# Análise do aproveitamento dos estudantes da Universidade do Estado do Amazonas
> Aluna: Juliany Raiol

## Introdução

Este dataset contém informações sobre os alunos do que cursaram matérias ofertadas durante o período de 2013 - 2014 na Universidade do Estado do Amazonas (UEA). Inicialmente, considerou-se que havia apenas dados referentes à matérias do ciclo básico, no entanto, após o processo de análise observou-se que há também informações de alunos veteranos.

O objetivo dessa análise é poder verificar quais matérias possuem maior índice de reprovação, bem como, descobrir se há uma ligação entre a origem do aluno e seu desempenho acadêmico.

## Considerações que deseja-se responder com esta análise

1. É possível considerar que alunos provenientes de municípios distantes da capital possuem desempenho menor que outros?
2. Qual a faixa etária dos alunos que entram na universidade por meio do SAES (antigo SIS)?
3. Há muitos alunos veteranos que ainda cursam matérias consideradas do ciclo básico?
4. A média de reprovação qual nos cursos de Análise e Desenvolvimento de Sistemas e Sistemas de Informação é menor que em cursos de Engenharia?

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
dataframe = pd.read_csv('../data/lyceum_2013_2014.csv')
dataframe.head()

,aluno,municipio_nascimento,dt_nasc,id_curso,nome_curso,tipo_ingresso,ano_ingresso,unidade_fisica,serie,id_disciplina,nome_disciplina,nota_final,situacao_hist,perc_presenca,faltas
0,1,Manaus,23/07/1995,EST17KSN,Análise e Desenvolvimento de Sistemas,SAES,2013,EST,1.0,ESTTPD108,Algoritmos e Programação,5.17,Rep Nota,0.9200,6.0
1,2,Manaus,23/07/1995,EST17KSN,Análise e Desenvolvimento de Sistemas,SAES,2013,EST,1.0,ESTBAS006,Comunicação e Expressão,6.00,Aprovado,1.0000,-1.0
2,3,Manaus,23/07/1995,EST17KSN,Análise e Desenvolvimento de Sistemas,SAES,2013,EST,1.0,ESTBAS002,Cálculo I,2.9,Rep Nota,1.0000,0.0
3,4,Manaus,23/07/1995,EST17KSN,Análise e Desenvolvimento de Sistemas,SAES,2013,EST,1.0,ESTBAS035,Empreendedorismo,8.50,Aprovado,1.0000,-1.0
4,5,Manaus,23/07/1995,EST17KSN,Análise e Desenvolvimento de Sistemas,SAES,2013,EST,1.0,ESTTPD100,Introdução à Computação,7.70,Aprovado,0.8667,8.0


## Características dos atributos

Os atributos encontrados nesse dataset foram classificados como numérico, norminal e ordinal. Pode-se perceber que a quantidade de atributos nominais é maior que os demais tipos.

Há cerca de 57075 linhas e 15 colunas. As colunas compreendem: <i> aluno</i>, <i>municipio_nascimento</i>, <i>dt_nasc</i>, <i>id_curso</i>, <i>nome_curso</i>, <i>tipo_ingresso</i>, <i>ano_ingresso</i>, <i>unidade_fisica</i>, <i>serie</i>, <i>id_disciplina</i>, <i>nome_disciplina</i>, <i>nota_final</i>, <i>situacao_hist</i>, <i>perc_presenca</i> e <i>faltas</i>.



| Nome da Coluna     |Classificação  |
|:------------------:|:-------------:|
| aluno              |  numérico     |
|municipio_nascimento|  nominal      |  
| dt_nasc            |  nominal      |
| id_curso           |  nominal      |
| nome_curso         |  nominal      |
| tipo_ingresso      |  nominal      |
| ano_ingresso       |  numérico     |
| unidade_fisica     |  nominal      |
| serie              |  ordinais     |
| id_disciplina      |  nominal      |
| nome_disciplina    |  nominal      |
| nota final         |  nominal      |
| situacao_hist      |  nominal      |
| perc_presenca      |  numérico     |
| faltas             |  numérico     |

## Qualidade dos dados

<strong> Precisão: </strong> Os dados mostraram-se imprecisos em certas colunas como, por exemplo, em <i> faltas </i> e <i> perc_presenca</i>, havia situações em que os devidos valores não foram cadastrados, alunos que não possuíam faltas lançadas e percentual de presença era nulo. Em certos casos, os que tinham essas características eram aqueles que foram <i> dispensados</i> da disciplina ou haviam <i>trancado</i> a matéria. Em <i>nota final</i>, em determinados campos, ao invés de nulo, encontrava-se apenas espaço em branco (' ').

<strong>Precisão semântica: </strong> Nas colunas de <i> faltas </i> e <i> perc_presenca</i>, os valore estavam como string e precisariam estar como ponto flutuante. Logo, fora do alcance da precisão semântica.

<strong> Precisão sintática: </strong> Nas coluna de <i>nota final </i>, havia valores representados por espaços vazios (' ').

<strong>Completude</strong>: Havia valores em falta, no entanto, após formação o dataset contém as informações necessárias para análise.

<strong>Pontualidade:</strong> Os dados não são tão antigos. Compreendem o período de 2013 - 2014.

## Visualização dos dados

In [6]:
dataframe.groupby('municipio_nascimento').size()

municipio_nascimento
Abrantes                       7
Acupe                         18
Ademar De Barros              18
Adustina                      18
Agisse                        81
Alenquer                     165
Alvaraes                      30
Amatura                       17
Andradina                     44
Anori                         38
Arequipa                      23
Ariquemes                     58
Autazes                       37
Axixa Do Tocantins            87
Bacabal                       22
Barcelos                      77
Barreirinha                    9
Belo Horizonte                14
Belém                        532
Benjamin Constant            167
Boa Vista                    126
Boa Vista Do Ramos             5
Boca Do Acre                 127
Borba                         17
Botucatu                      18
Brasilia                      29
Brejo Santo                   11
Caico                         80
Camocim                       41
Campina Grande        